# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-ad886f7aca-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)


# GEO Workflow Series ("American Architects") 

Consider the following exploratory information need:

> You want to study the most prolific american architects

## Useful URIs for the current workflow


The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P279`    | subclass      | predicate |
| `wdt:P106`    | occupation    | predicate |
| `wdt:P17`     | country       | predicate |
| `wdt:P27`     | citizenship   | predicate |
| `wd:Q5604`    | Frank Lloyd Wright     | node      |
| `wd:Q30`      | U.S.A.        | node |
| `wd:Q42973`   | architect     | node |
| `wd:Q2081276` | Pettit Memorial Chapel        | node |
| `wd:Q2977`    | Cathedral     | node |



Also consider

```
?p wdt:P27 wd:Q30  . 
?p wdt:P106 wd:Q42973  . 
```

is the BGP to retrieve all **american architects**

## Workload Goals

1. Identify the BGP to obtain the notable works of an architect and their location

2. Identify the BGP that connect a building with its architect, type, and architectural style

3. Which architect is the most prolific in the U.S. and outside?

4. Analyze the location of the buildings designed by the american architects
 
   4.1 Which styles exists in the U.S.? Which type of building?
   
   4.2 In which U.S. states there are more notable buildings?
   
   4.3 Which U.S. state contains the largest number of building designed by U.S. architects


In [2]:
# start your workflow here

In [3]:
queryString = """
SELECT COUNT(*)
WHERE { 
    ?p wdt:P27 wd:Q30  . 
    ?p wdt:P106 wd:Q42973  . 
} 
"""
print("Results")
run_query(queryString)

Results
[('callret-0', '3354')]


1

#### Identify the BGP to obtain the notable works of an architect and their location

In [4]:
# american architects human readable:
queryString = """
SELECT *
WHERE { 
    ?p wdt:P27 wd:Q30  . 
    ?p wdt:P106 wd:Q42973  . 
    ?p <http://schema.org/name> ?name .
} 
LIMIT 10
"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q13647155'), ('name', 'Marshall Strabala')]
[('p', 'http://www.wikidata.org/entity/Q15463168'), ('name', 'John Mead Howells')]
[('p', 'http://www.wikidata.org/entity/Q16007786'), ('name', 'Roscoe DeWitt')]
[('p', 'http://www.wikidata.org/entity/Q16008039'), ('name', 'Herbert B. Hunter')]
[('p', 'http://www.wikidata.org/entity/Q16008282'), ('name', 'Mott B. Schmidt')]
[('p', 'http://www.wikidata.org/entity/Q16010946'), ('name', 'Henry Powell Hopkins')]
[('p', 'http://www.wikidata.org/entity/Q16012978'), ('name', 'Edward Giddings')]
[('p', 'http://www.wikidata.org/entity/Q16013299'), ('name', 'Max M. Sandfield')]
[('p', 'http://www.wikidata.org/entity/Q16017336'), ('name', 'Richard Lewis Blinder')]
[('p', 'http://www.wikidata.org/entity/Q16023299'), ('name', 'Ambrose J. Russell')]


10

In [5]:
# returning all the american architectures:
queryString = """
SELECT ?americanar ?americanarLabel ?p ?pLabel ?o ?oLabel
WHERE { 
    ?americanar wdt:P27 wd:Q30  . 
    ?americanar wdt:P106 wd:Q42973 ;
          ?p ?o .       
} 
LIMIT 50

"""
print("Results")
run_query(queryString)

Results
[('americanar', 'http://www.wikidata.org/entity/Q13647155'), ('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('o', 'http://wikiba.se/ontology#Item')]
[('americanar', 'http://www.wikidata.org/entity/Q15463168'), ('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('o', 'http://wikiba.se/ontology#Item')]
[('americanar', 'http://www.wikidata.org/entity/Q16007786'), ('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('o', 'http://wikiba.se/ontology#Item')]
[('americanar', 'http://www.wikidata.org/entity/Q16008039'), ('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('o', 'http://wikiba.se/ontology#Item')]
[('americanar', 'http://www.wikidata.org/entity/Q16008282'), ('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('o', 'http://wikiba.se/ontology#Item')]
[('americanar', 'http://www.wikidata.org/entity/Q16010946'), ('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('o', 'http://wikiba.se/ontology#Item')]
[('americanar', 'http://www.wiki

50

In [6]:
# the architects names searching
queryString = """
SELECT DISTINCT ?americanar ?americanarLabel 
WHERE { 
    ?americanar wdt:P27 wd:Q30  . 
    ?americanar wdt:P106 wd:Q42973 .
    ?americanar <http://schema.org/name> ?americanarLabel . 
} 
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('americanar', 'http://www.wikidata.org/entity/Q13647155'), ('americanarLabel', 'Marshall Strabala')]
[('americanar', 'http://www.wikidata.org/entity/Q15463168'), ('americanarLabel', 'John Mead Howells')]
[('americanar', 'http://www.wikidata.org/entity/Q16007786'), ('americanarLabel', 'Roscoe DeWitt')]
[('americanar', 'http://www.wikidata.org/entity/Q16008039'), ('americanarLabel', 'Herbert B. Hunter')]
[('americanar', 'http://www.wikidata.org/entity/Q16008282'), ('americanarLabel', 'Mott B. Schmidt')]
[('americanar', 'http://www.wikidata.org/entity/Q16010946'), ('americanarLabel', 'Henry Powell Hopkins')]
[('americanar', 'http://www.wikidata.org/entity/Q16012978'), ('americanarLabel', 'Edward Giddings')]
[('americanar', 'http://www.wikidata.org/entity/Q16013299'), ('americanarLabel', 'Max M. Sandfield')]
[('americanar', 'http://www.wikidata.org/entity/Q16017336'), ('americanarLabel', 'Richard Lewis Blinder')]
[('americanar', 'http://www.wikidata.org/entity/Q16023299'), ('amer

50

In [7]:
# 1. returning their works (building names)
# 2. find their arts (buildings)
# 0.  'petit memorial chapel'
# search for the location
queryString = """
SELECT ?p ?pname ?o ?oname WHERE{
  ?p ?o wd:Q2081276 .
  ## labels
  ?p <http://schema.org/name> ?pname .
  ?o <http://schema.org/name> ?oname .
}
"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q55276305'), ('pname', 'Category:Pettit Memorial Chapel'), ('o', 'http://www.wikidata.org/prop/direct/P301'), ('oname', "category's main topic")]
[('p', 'http://www.wikidata.org/entity/Q5604'), ('pname', 'Frank Lloyd Wright'), ('o', 'http://www.wikidata.org/prop/direct/P800'), ('oname', 'notable work')]


2

In [8]:
# search for objects related to this node
queryString = """
SELECT ?p ?pLabel ?o ?oLabel WHERE{
  wd:Q2081276 ?p ?o .
  
    ## labels
  ?p <http://schema.org/name> ?pLabel .
  ?o <http://schema.org/name> ?oLabel .
  
}
"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P149'), ('pLabel', 'architectural style'), ('o', 'http://www.wikidata.org/entity/Q245188'), ('oLabel', 'modern architecture')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pLabel', 'country'), ('o', 'http://www.wikidata.org/entity/Q30'), ('oLabel', 'United States of America')]
[('p', 'http://www.wikidata.org/prop/direct/P1435'), ('pLabel', 'heritage designation'), ('o', 'http://www.wikidata.org/entity/Q19558910'), ('oLabel', 'National Register of Historic Places listed place')]
[('p', 'http://www.wikidata.org/prop/direct/P84'), ('pLabel', 'architect'), ('o', 'http://www.wikidata.org/entity/Q5604'), ('oLabel', 'Frank Lloyd Wright')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pLabel', 'located in the administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q185609'), ('oLabel', 'Belvidere')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pLabel', "topic's main category"), ('o', 'http://www.wik

7

In [9]:

queryString = """
SELECT ?p ?pLabel ?o ?oLabel WHERE{
  wd:Q2081276 ?p ?o .
  ## labels
     ?o <http://schema.org/name> ?oLabel .
}
"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P149'), ('o', 'http://www.wikidata.org/entity/Q245188'), ('oLabel', 'modern architecture')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('o', 'http://www.wikidata.org/entity/Q30'), ('oLabel', 'United States of America')]
[('p', 'http://www.wikidata.org/prop/direct/P1435'), ('o', 'http://www.wikidata.org/entity/Q19558910'), ('oLabel', 'National Register of Historic Places listed place')]
[('p', 'http://www.wikidata.org/prop/direct/P84'), ('o', 'http://www.wikidata.org/entity/Q5604'), ('oLabel', 'Frank Lloyd Wright')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('o', 'http://www.wikidata.org/entity/Q185609'), ('oLabel', 'Belvidere')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('o', 'http://www.wikidata.org/entity/Q55276305'), ('oLabel', 'Category:Pettit Memorial Chapel')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('o', 'http://www.wikidata.org/entity/Q108325'), ('oLabel', 'chapel')]


7

In [10]:
# search for the location and geonames

queryString = """
SELECT ?p ?pLabel ?o ?oLabel WHERE{
  wd:Q2081276 ?p ?o .
    ## labels
     ?p <http://schema.org/name> ?pLabel .
     ?o <http://schema.org/name> ?oLabel .

}
"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P149'), ('pLabel', 'architectural style'), ('o', 'http://www.wikidata.org/entity/Q245188'), ('oLabel', 'modern architecture')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pLabel', 'country'), ('o', 'http://www.wikidata.org/entity/Q30'), ('oLabel', 'United States of America')]
[('p', 'http://www.wikidata.org/prop/direct/P1435'), ('pLabel', 'heritage designation'), ('o', 'http://www.wikidata.org/entity/Q19558910'), ('oLabel', 'National Register of Historic Places listed place')]
[('p', 'http://www.wikidata.org/prop/direct/P84'), ('pLabel', 'architect'), ('o', 'http://www.wikidata.org/entity/Q5604'), ('oLabel', 'Frank Lloyd Wright')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pLabel', 'located in the administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q185609'), ('oLabel', 'Belvidere')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pLabel', "topic's main category"), ('o', 'http://www.wik

7

In [11]:
# wd:Q19558910
# wdt:P131 
#  wd:Q185609	Belvidere is an instance of wdt:P131 
# arts
# (architect) wdt:P84	http://www.wikidata.org/prop/direct/P84	 wd:Q5604	Frank Lloyd Wright (instance of that properties)
# chapel is connected to the artist

queryString = """
SELECT ?p ?pLabel ?o ?oLabel WHERE{
  wd:Q2081276 ?p ?o .
    ## labels
     ?p <http://schema.org/name> ?pLabel .
     ?o <http://schema.org/name> ?oLabel .
}
"""
print("Results")
run_query(queryString)

# wdt:P84	http://www.wikidata.org/prop/direct/P84	 wd:Q5604	Frank Lloyd Wright
# architect(wdt:P84) is wd:Q5604 

Results
[('p', 'http://www.wikidata.org/prop/direct/P149'), ('pLabel', 'architectural style'), ('o', 'http://www.wikidata.org/entity/Q245188'), ('oLabel', 'modern architecture')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pLabel', 'country'), ('o', 'http://www.wikidata.org/entity/Q30'), ('oLabel', 'United States of America')]
[('p', 'http://www.wikidata.org/prop/direct/P1435'), ('pLabel', 'heritage designation'), ('o', 'http://www.wikidata.org/entity/Q19558910'), ('oLabel', 'National Register of Historic Places listed place')]
[('p', 'http://www.wikidata.org/prop/direct/P84'), ('pLabel', 'architect'), ('o', 'http://www.wikidata.org/entity/Q5604'), ('oLabel', 'Frank Lloyd Wright')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pLabel', 'located in the administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q185609'), ('oLabel', 'Belvidere')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pLabel', "topic's main category"), ('o', 'http://www.wik

7

In [12]:
## this shows the works of Frank Lloyd Wright artist
queryString = """

SELECT DISTINCT ?work ?workLabel 
WHERE { 
    ?work wdt:P84 wd:Q5604 .
    ## labels
     ?work <http://schema.org/name> ?workLabel  .
} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('work', 'http://www.wikidata.org/entity/Q929965'), ('workLabel', 'Robie House')]
[('work', 'http://www.wikidata.org/entity/Q2081276'), ('workLabel', 'Pettit Memorial Chapel')]
[('work', 'http://www.wikidata.org/entity/Q965011'), ('workLabel', 'Millard House')]
[('work', 'http://www.wikidata.org/entity/Q201469'), ('workLabel', 'Solomon R. Guggenheim Museum')]
[('work', 'http://www.wikidata.org/entity/Q2424445'), ('workLabel', 'Thomas H. Gale House')]
[('work', 'http://www.wikidata.org/entity/Q4956682'), ('workLabel', 'Brandes House')]
[('work', 'http://www.wikidata.org/entity/Q5468984'), ('workLabel', 'Forest House')]
[('work', 'http://www.wikidata.org/entity/Q6531634'), ('workLabel', 'Robert M. Lamp House')]
[('work', 'http://www.wikidata.org/entity/Q7010799'), ('workLabel', 'New Orleans Union Station')]
[('work', 'http://www.wikidata.org/entity/Q64683561'), ('workLabel', 'Sherman M. Booth House')]
[('work', 'http://www.wikidata.org/entity/Q7296752'), ('workLabel', 'Ravine Bl

50

In [13]:
## time. 
# wd:Q929965
queryString = """
SELECT DISTINCT ?p ?pname
#COUNT (DISTINCT ?p)
WHERE { 

    wd:Q929965 ?p ?o .
    # Labels
    ?p <http://schema.org/name> ?pname .
} 
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P7958'), ('pname', 'Chicago Landmarks ID')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pname', 'located in the administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('pname', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1435'), ('pname', 'heritage designation')]
[('p', 'http://www.wikidata.org/prop/direct/P149'), ('pname', 'architectural style')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('pname', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P2046'), ('pname', 'area')]
[('p', 'http://www.wikidata.org/prop/direct/P214'), ('pname', 'VIAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pname', 'location')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P3219'), ('pname', 'Encyclop

22

## Illustrate the names of the artists and their works

In [14]:
## list all of the arts and the buildings
## this shows the works of all american artists with their names 


queryString = """

SELECT ?americanarLabel (GROUP_CONCAT(DISTINCT ?workLabel; SEPARATOR=", ") AS ?workLabels)
WHERE { 
    ?americanar wdt:P27 wd:Q30  . 
    ?americanar wdt:P106 wd:Q42973 .
    ?work wdt:P84 ?americanar .
        ## labels
     ?americanar <http://schema.org/name> ?americanarLabel .
     ?work <http://schema.org/name> ?workLabel .
} 
LIMIT 50

"""

print("Results")
run_query(queryString)

Results
[('americanarLabel', 'Nathan Clifford Ricker'), ('workLabels', 'Altgeld Hall')]
[('americanarLabel', 'Donn Barber'), ('workLabels', 'Hartford Times Building, Terminal Station, Travelers Tower')]
[('americanarLabel', 'Francis Costigan'), ('workLabels', 'Charles L. Shrewsbury House, Lanier Mansion, Le Méridien Indianapolis Hotel')]
[('americanarLabel', 'Albert C. Martin Jr.'), ('workLabels', '1900 Avenue of the Stars')]
[('americanarLabel', 'Frank E. Edbrooke'), ('workLabels', 'Central Presbyterian Church, Joslin Dry Goods Company Building, Masonic Temple Building, Steamboat Springs Depot, Temple Emanuel, The Brown Palace Hotel and Spa, Autograph Collection')]
[('americanarLabel', 'Félix Candela'), ('workLabels', "L'Oceanogràfic, Palacio de los Deportes, church of Nuestra Señora de Guadalupe")]
[('americanarLabel', 'James R. Edmunds, Jr.'), ('workLabels', 'Eastern High School, Maryland Stadium')]
[('americanarLabel', 'Gottlieb Schumacher'), ('workLabels', 'Carmel Winery')]
[('ame

50

In [15]:
## list all of the arts and the buildings
## this shows the works of all american artists with their names 


queryString = """

SELECT ?americanar ?americanarLabel ?work ?workLabel 
WHERE { 
    ?americanar wdt:P27 wd:Q30  . 
    ?americanar wdt:P106 wd:Q42973 .
    ?work wdt:P84 ?americanar .
        ## labels
     ?americanar <http://schema.org/name> ?americanarLabel .
     ?work <http://schema.org/name> ?workLabel .
} 
LIMIT 50

"""

print("Results")
run_query(queryString)

Results
[('americanar', 'http://www.wikidata.org/entity/Q3017360'), ('americanarLabel', 'David Adler'), ('work', 'http://www.wikidata.org/entity/Q16198500'), ('workLabel', 'Mrs. Isaac D. Adler House')]
[('americanar', 'http://www.wikidata.org/entity/Q5006523'), ('americanarLabel', 'C. Howard Crane'), ('work', 'http://www.wikidata.org/entity/Q17040485'), ('workLabel', 'World Theater')]
[('americanar', 'http://www.wikidata.org/entity/Q5077062'), ('americanarLabel', 'Charles E. Choate'), ('work', 'http://www.wikidata.org/entity/Q17059514'), ('workLabel', 'Tennille Banking Company Building')]
[('americanar', 'http://www.wikidata.org/entity/Q5540911'), ('americanarLabel', 'George I. Barnett'), ('work', 'http://www.wikidata.org/entity/Q17299881'), ('workLabel', 'Samuel Moody Grubbs House')]
[('americanar', 'http://www.wikidata.org/entity/Q15437942'), ('americanarLabel', 'Henry Franklin Kilburn'), ('work', 'http://www.wikidata.org/entity/Q18151982'), ('workLabel', 'Pabst Hotel')]
[('americana

50

## Demonstrate the location of their works

In [16]:
# a country 
# because it might be a case that the artist is american, but the building he/she made is in another country
queryString = """



SELECT ?p ?pLabel ?o ?oLabel WHERE{
  wd:Q2081276 ?p ?o .
      ## labels
     ?p <http://schema.org/name> ?pLabel .
     ?o <http://schema.org/name> ?oLabel .
}


"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P149'), ('pLabel', 'architectural style'), ('o', 'http://www.wikidata.org/entity/Q245188'), ('oLabel', 'modern architecture')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pLabel', 'country'), ('o', 'http://www.wikidata.org/entity/Q30'), ('oLabel', 'United States of America')]
[('p', 'http://www.wikidata.org/prop/direct/P1435'), ('pLabel', 'heritage designation'), ('o', 'http://www.wikidata.org/entity/Q19558910'), ('oLabel', 'National Register of Historic Places listed place')]
[('p', 'http://www.wikidata.org/prop/direct/P84'), ('pLabel', 'architect'), ('o', 'http://www.wikidata.org/entity/Q5604'), ('oLabel', 'Frank Lloyd Wright')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pLabel', 'located in the administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q185609'), ('oLabel', 'Belvidere')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pLabel', "topic's main category"), ('o', 'http://www.wik

7

In [17]:
# from above results, these two related to location are shown
# its city
# wdt:P131	http://www.wikidata.org/prop/direct/P131	 wd:Q185609	Belvidere

# find the locations of the works by Frank Lloyd Wright 

queryString = """
    SELECT ?work ?workLabel ?o ?oLabel  WHERE{
    
    ## this shows the works of Frank Lloyd Wright artist

      ?work wdt:P84 wd:Q5604 .
      wd:Q2081276 ?p ?o .
      wd:Q2081276 wdt:P131 ?o .
      
     ## labels
     ?work <http://schema.org/name> ?workLabel .
     ?o <http://schema.org/name> ?oLabel .
    }
    LIMIT 50

"""

print("Results")
run_query(queryString)

#### NO USEFUL RESULT #####

Results
[('work', 'http://www.wikidata.org/entity/Q929965'), ('workLabel', 'Robie House'), ('o', 'http://www.wikidata.org/entity/Q185609'), ('oLabel', 'Belvidere')]
[('work', 'http://www.wikidata.org/entity/Q2081276'), ('workLabel', 'Pettit Memorial Chapel'), ('o', 'http://www.wikidata.org/entity/Q185609'), ('oLabel', 'Belvidere')]
[('work', 'http://www.wikidata.org/entity/Q965011'), ('workLabel', 'Millard House'), ('o', 'http://www.wikidata.org/entity/Q185609'), ('oLabel', 'Belvidere')]
[('work', 'http://www.wikidata.org/entity/Q201469'), ('workLabel', 'Solomon R. Guggenheim Museum'), ('o', 'http://www.wikidata.org/entity/Q185609'), ('oLabel', 'Belvidere')]
[('work', 'http://www.wikidata.org/entity/Q2424445'), ('workLabel', 'Thomas H. Gale House'), ('o', 'http://www.wikidata.org/entity/Q185609'), ('oLabel', 'Belvidere')]
[('work', 'http://www.wikidata.org/entity/Q4956682'), ('workLabel', 'Brandes House'), ('o', 'http://www.wikidata.org/entity/Q185609'), ('oLabel', 'Belvidere')]
[('wor

50

In [18]:

## this shows the works of Frank Lloyd Wright artist with their locations (just the city of their location)


queryString = """

    SELECT DISTINCT ?work ?workLabel ?city ?cityLabel
    WHERE { 
        ?work wdt:P84 wd:Q5604 .
         ?work wdt:P131 ?city .
         
     ## labels
     ?work <http://schema.org/name> ?workLabel .
     ?city <http://schema.org/name> ?cityLabel .
    } 
    LIMIT 50

"""

print("Results")
run_query(queryString)

Results
[('work', 'http://www.wikidata.org/entity/Q5088241'), ('workLabel', 'Chauncey L. Williams Residence'), ('city', 'http://www.wikidata.org/entity/Q2299339'), ('cityLabel', 'River Forest')]
[('work', 'http://www.wikidata.org/entity/Q6077750'), ('workLabel', 'Isabel Roberts House'), ('city', 'http://www.wikidata.org/entity/Q2299339'), ('cityLabel', 'River Forest')]
[('work', 'http://www.wikidata.org/entity/Q7641695'), ('workLabel', 'Suntop Homes'), ('city', 'http://www.wikidata.org/entity/Q1132213'), ('cityLabel', 'Ardmore')]
[('work', 'http://www.wikidata.org/entity/Q7456571'), ('workLabel', 'Seth Peterson Cottage'), ('city', 'http://www.wikidata.org/entity/Q1570118'), ('cityLabel', 'Reedsburg')]
[('work', 'http://www.wikidata.org/entity/Q7291537'), ('workLabel', 'Randall Fawcett House'), ('city', 'http://www.wikidata.org/entity/Q590932'), ('cityLabel', 'Los Baños')]
[('work', 'http://www.wikidata.org/entity/Q8072136'), ('workLabel', 'Zimmerman House'), ('city', 'http://www.wikida

50

In [19]:

## this shows the works of Frank Lloyd Wright artist with their locations(just the city of their location)
queryString = """

    SELECT ?work ?workLabel ?city ?cityLabel
    WHERE { 
        ?work wdt:P84 wd:Q5604 .
         ?work wdt:P131 ?city .
             ## labels
     ?work <http://schema.org/name> ?workLabel .
     ?city <http://schema.org/name> ?cityLabel .
    } 
    LIMIT 20

"""

print("Results")
run_query(queryString)

Results
[('work', 'http://www.wikidata.org/entity/Q5088241'), ('workLabel', 'Chauncey L. Williams Residence'), ('city', 'http://www.wikidata.org/entity/Q2299339'), ('cityLabel', 'River Forest')]
[('work', 'http://www.wikidata.org/entity/Q6077750'), ('workLabel', 'Isabel Roberts House'), ('city', 'http://www.wikidata.org/entity/Q2299339'), ('cityLabel', 'River Forest')]
[('work', 'http://www.wikidata.org/entity/Q7641695'), ('workLabel', 'Suntop Homes'), ('city', 'http://www.wikidata.org/entity/Q1132213'), ('cityLabel', 'Ardmore')]
[('work', 'http://www.wikidata.org/entity/Q7456571'), ('workLabel', 'Seth Peterson Cottage'), ('city', 'http://www.wikidata.org/entity/Q1570118'), ('cityLabel', 'Reedsburg')]
[('work', 'http://www.wikidata.org/entity/Q7291537'), ('workLabel', 'Randall Fawcett House'), ('city', 'http://www.wikidata.org/entity/Q590932'), ('cityLabel', 'Los Baños')]
[('work', 'http://www.wikidata.org/entity/Q8072136'), ('workLabel', 'Zimmerman House'), ('city', 'http://www.wikida

20

# Identify the BGP to obtain the notable works of an architect and their location.

In [20]:
## final 
# [('p', 'http://www.wikidata.org/prop/direct/P7958'), ('pname', 'Chicago Landmarks ID')]
# [('p', 'http://www.wikidata.org/prop/direct/P131'), ('pname', 'located in the administrative territorial entity')]
# [('p', 'http://www.wikidata.org/prop/direct/P149'), ('pname', 'architectural style')]
# [('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]
# [('p', 'http://www.wikidata.org/prop/direct/P2046'), ('pname', 'area')]
# [('p', 'http://www.wikidata.org/prop/direct/P276'), ('pname', 'location')]
# [('p', 'http://www.wikidata.org/prop/direct/P361'), ('pname', 'part of')]
# [('p', 'http://www.wikidata.org/prop/direct/P84'), ('pname', 'architect')]
# [('p', 'http://www.wikidata.org/prop/direct/P6375'), ('pname', 'street address')]

queryString = """

    SELECT ?americanar ?americanarLabel ?work ?workLabel ?cityLabel
    WHERE { 
        ?americanar wdt:P27 wd:Q30  . 
        ?americanar wdt:P106 wd:Q42973 .
        ?work wdt:P84 ?americanar ;
              wdt:P131 ?city .
            ## labels
     ?americanar <http://schema.org/name> ?americanarLabel .
     ?work <http://schema.org/name> ?workLabel .
     ?city <http://schema.org/name> ?cityLabel .
    } 
    LIMIT 40
    
"""

print("Results")
run_query(queryString)

Results
[('americanar', 'http://www.wikidata.org/entity/Q29093464'), ('americanarLabel', 'Ivey P. Crutchfield'), ('work', 'http://www.wikidata.org/entity/Q29093460'), ('workLabel', 'Peterson-Wilbanks House'), ('cityLabel', 'Vidalia')]
[('americanar', 'http://www.wikidata.org/entity/Q29093464'), ('americanarLabel', 'Ivey P. Crutchfield'), ('work', 'http://www.wikidata.org/entity/Q29093503'), ('workLabel', 'Leader–Rosansky House'), ('cityLabel', 'Vidalia')]
[('americanar', 'http://www.wikidata.org/entity/Q29093464'), ('americanarLabel', 'Ivey P. Crutchfield'), ('work', 'http://www.wikidata.org/entity/Q28811758'), ('workLabel', 'Crawford W. Brazell House'), ('cityLabel', 'Vidalia')]
[('americanar', 'http://www.wikidata.org/entity/Q5077062'), ('americanarLabel', 'Charles E. Choate'), ('work', 'http://www.wikidata.org/entity/Q16995712'), ('workLabel', 'Holt Brothers Banking Company Building'), ('cityLabel', 'Sandersville')]
[('americanar', 'http://www.wikidata.org/entity/Q17626642'), ('amer

40

In [21]:
## just # LABELS 

queryString = """

    SELECT ?americanarLabel ?workLabel ?cityLabel
    WHERE { 
        ?americanar wdt:P27 wd:Q30  . 
        ?americanar wdt:P106 wd:Q42973 .
        ?work wdt:P84 ?americanar ;
              wdt:P131 ?city .
            ## labels
     ?americanar <http://schema.org/name> ?americanarLabel .
     ?work <http://schema.org/name> ?workLabel .
     ?city <http://schema.org/name> ?cityLabel .

    } 
    LIMIT 40
    
"""

print("Results")
run_query(queryString)


Results
[('americanarLabel', 'Ivey P. Crutchfield'), ('workLabel', 'Peterson-Wilbanks House'), ('cityLabel', 'Vidalia')]
[('americanarLabel', 'Ivey P. Crutchfield'), ('workLabel', 'Leader–Rosansky House'), ('cityLabel', 'Vidalia')]
[('americanarLabel', 'Ivey P. Crutchfield'), ('workLabel', 'Crawford W. Brazell House'), ('cityLabel', 'Vidalia')]
[('americanarLabel', 'Charles E. Choate'), ('workLabel', 'Holt Brothers Banking Company Building'), ('cityLabel', 'Sandersville')]
[('americanarLabel', 'Francis Willford Fitzpatrick'), ('workLabel', 'Johnston Memorial Building'), ('cityLabel', 'Wallace')]
[('americanarLabel', 'C. Howard Crane'), ('workLabel', 'World Theater'), ('cityLabel', 'Omaha')]
[('americanarLabel', 'Abraham Wesley Eager'), ('workLabel', 'Auditorium (Torrance High School)'), ('cityLabel', 'Torrance')]
[('americanarLabel', 'Charles E. Choate'), ('workLabel', 'James Kelley House'), ('cityLabel', 'Washington County')]
[('americanarLabel', 'Ambrose J. Russell'), ('workLabel', '

40

In [22]:
## final 
# [('p', 'http://www.wikidata.org/prop/direct/P7958'), ('pname', 'Chicago Landmarks ID')]
# [('p', 'http://www.wikidata.org/prop/direct/P131'), ('pname', 'located in the administrative territorial entity')]
# [('p', 'http://www.wikidata.org/prop/direct/P149'), ('pname', 'architectural style')]
# [('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]
# [('p', 'http://www.wikidata.org/prop/direct/P2046'), ('pname', 'area')]
# [('p', 'http://www.wikidata.org/prop/direct/P276'), ('pname', 'location')]
# [('p', 'http://www.wikidata.org/prop/direct/P361'), ('pname', 'part of')]
# [('p', 'http://www.wikidata.org/prop/direct/P84'), ('pname', 'architect')]
# [('p', 'http://www.wikidata.org/prop/direct/P6375'), ('pname', 'street address')]

queryString = """
SELECT ?americanarLabel (GROUP_CONCAT(DISTINCT ?workLabel; SEPARATOR=", ") AS ?workLabels)

    WHERE { 
        ?americanar wdt:P27 wd:Q30  . 
        ?americanar wdt:P106 wd:Q42973 .
        ?work wdt:P84 ?americanar ;
              wdt:P131 ?city ;
              wdt:P17 ?country ;
              wdt:P6375 ?street_address .
              
            ## labels
     ?americanar <http://schema.org/name> ?americanarLabel .
     ?work <http://schema.org/name> ?workLabel .
     ?city <http://schema.org/name> ?cityLabel .
     ?country <http://schema.org/name> ?countryname .

    } 
    LIMIT 10
    
"""

print("Results")
run_query(queryString)

Results
[('americanarLabel', 'Irving Gill'), ('workLabels', 'Miltimore House, Oceanside City Hall and Fire Station')]
[('americanarLabel', 'Charles Follen McKim'), ('workLabels', 'Boston Public Library, McKim Building, Schomburg Center for Research in Black Culture, Schomburg Center for Research in Black Culture (original building), The Morgan Library & Museum')]
[('americanarLabel', 'Appleton P. Clark, Jr.'), ('workLabels', 'U.S. Civil Service Commission Building')]
[('americanarLabel', 'J. W. Golucke'), ('workLabels', 'Johnson County Courthouse, Old Union County Courthouse')]
[('americanarLabel', 'Anthony J. Lumsden'), ('workLabels', 'Little Tokyo Branch Library')]
[('americanarLabel', 'Frank Weary'), ('workLabels', 'The Gothic Building')]
[('americanarLabel', 'Mifflin E. Bell'), ('workLabels', 'Monroe County Courthouse')]
[('americanarLabel', 'Henry L. Newhouse'), ('workLabels', 'Hotel Trenier')]
[('americanarLabel', 'James Renwick Jr.'), ('workLabels', 'Oak Hill Cemetery, Oak Hill 

10

## Identify the BGP that connect a building with its architect, type, and architectural style

In [23]:

queryString = """

    SELECT ?americanar ?americanarLabel ?work ?workLabel 
    WHERE { 
        ?work wdt:P84 ?americanar .
        ?americanar wdt:P27 wd:Q30  . 
        ?americanar wdt:P106 wd:Q42973 .
        
            ## labels
     ?americanar <http://schema.org/name> ?americanarLabel .
     ?work <http://schema.org/name> ?workLabel  .
    } 
    LIMIT 50
    
"""

print("Results")
run_query(queryString)

Results
[('americanar', 'http://www.wikidata.org/entity/Q3017360'), ('americanarLabel', 'David Adler'), ('work', 'http://www.wikidata.org/entity/Q16198500'), ('workLabel', 'Mrs. Isaac D. Adler House')]
[('americanar', 'http://www.wikidata.org/entity/Q5006523'), ('americanarLabel', 'C. Howard Crane'), ('work', 'http://www.wikidata.org/entity/Q17040485'), ('workLabel', 'World Theater')]
[('americanar', 'http://www.wikidata.org/entity/Q5077062'), ('americanarLabel', 'Charles E. Choate'), ('work', 'http://www.wikidata.org/entity/Q17059514'), ('workLabel', 'Tennille Banking Company Building')]
[('americanar', 'http://www.wikidata.org/entity/Q5540911'), ('americanarLabel', 'George I. Barnett'), ('work', 'http://www.wikidata.org/entity/Q17299881'), ('workLabel', 'Samuel Moody Grubbs House')]
[('americanar', 'http://www.wikidata.org/entity/Q15437942'), ('americanarLabel', 'Henry Franklin Kilburn'), ('work', 'http://www.wikidata.org/entity/Q18151982'), ('workLabel', 'Pabst Hotel')]
[('americana

50

In [24]:
# chapel 
queryString = """

    SELECT ?p ?pLabel ?o ?oLabel WHERE{
      wd:Q2081276 ?p ?o .
      
          ## labels
     ?p <http://schema.org/name> ?pLabel .
     ?o <http://schema.org/name> ?oLabel .

    }
    LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P149'), ('pLabel', 'architectural style'), ('o', 'http://www.wikidata.org/entity/Q245188'), ('oLabel', 'modern architecture')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pLabel', 'country'), ('o', 'http://www.wikidata.org/entity/Q30'), ('oLabel', 'United States of America')]
[('p', 'http://www.wikidata.org/prop/direct/P1435'), ('pLabel', 'heritage designation'), ('o', 'http://www.wikidata.org/entity/Q19558910'), ('oLabel', 'National Register of Historic Places listed place')]
[('p', 'http://www.wikidata.org/prop/direct/P84'), ('pLabel', 'architect'), ('o', 'http://www.wikidata.org/entity/Q5604'), ('oLabel', 'Frank Lloyd Wright')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pLabel', 'located in the administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q185609'), ('oLabel', 'Belvidere')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pLabel', "topic's main category"), ('o', 'http://www.wik

7

In [25]:
# Identify the BGP that connect Pettit Memorial Chapel with its type, and architectural style
# TYPE
# wdt:P31	http://www.wikidata.org/prop/direct/P31	 wd:Q108325	chapel
# architectural style
# wdt:P149	http://www.wikidata.org/prop/direct/P149	 wd:Q245188	modern architecture

queryString = """

    SELECT DISTINCT ?type ?typeLabel ?architectural_style ?architectural_styleLabel WHERE{
      wd:Q2081276 ?p ?o ;
                  wdt:P31 ?type ;
                  wdt:P149 ?architectural_style .    
      
          ## labels
     ?type <http://schema.org/name> ?typeLabel .
     ?architectural_style <http://schema.org/name> ?architectural_styleLabel .
      

    }
    LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('type', 'http://www.wikidata.org/entity/Q108325'), ('typeLabel', 'chapel'), ('architectural_style', 'http://www.wikidata.org/entity/Q245188'), ('architectural_styleLabel', 'modern architecture')]


1

In [26]:
# Identify the BGP that connect Pettit Memorial Chapel with its architect, type, and architectural style
# first find the TYPE
# wdt:P31	http://www.wikidata.org/prop/direct/P31	 wd:Q108325	chapel
# architectural style
# wdt:P149	http://www.wikidata.org/prop/direct/P149	 wd:Q245188	modern architecture


queryString = """

    SELECT DISTINCT ?typebuilding ?typebuildingLabel ?architectural_style ?architectural_styleLabel WHERE{
      wd:Q2081276 ?p ?o ;
                  wdt:P31 ?typebuilding ;
                  wdt:P149 ?architectural_style .    
    
    ## labels
     ?typebuilding <http://schema.org/name> ?typebuildingLabel .
     ?architectural_style <http://schema.org/name> ?architectural_styleLabel .

    }
     LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('typebuilding', 'http://www.wikidata.org/entity/Q108325'), ('typebuildingLabel', 'chapel'), ('architectural_style', 'http://www.wikidata.org/entity/Q245188'), ('architectural_styleLabel', 'modern architecture')]


1

In [27]:
# the works of the artist وits works and buildings

queryString = """


    SELECT ?p ?pLabel WHERE{
      wd:Q5604 ?p wd:Q2081276 .
    ## labels
     ?p <http://schema.org/name> ?pLabel .
    } 
    LIMIT 10


"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P800'), ('pLabel', 'notable work')]


1

In [28]:

queryString = """

    SELECT ?p ?pLabel ?has ?hasLabel ?work ?workLabel WHERE{
      wd:Q5604 ?p wd:Q2081276 .
      wd:Q2081276 ?has ?work .
      
          ## labels
     ?p <http://schema.org/name> ?pLabel .
     ?has <http://schema.org/name> ?hasLabel .
     ?work <http://schema.org/name> ?workLabel

    }
    LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P800'), ('pLabel', 'notable work'), ('has', 'http://www.wikidata.org/prop/direct/P17'), ('hasLabel', 'country'), ('work', 'http://www.wikidata.org/entity/Q30'), ('workLabel', 'United States of America')]
[('p', 'http://www.wikidata.org/prop/direct/P800'), ('pLabel', 'notable work'), ('has', 'http://www.wikidata.org/prop/direct/P1435'), ('hasLabel', 'heritage designation'), ('work', 'http://www.wikidata.org/entity/Q19558910'), ('workLabel', 'National Register of Historic Places listed place')]
[('p', 'http://www.wikidata.org/prop/direct/P800'), ('pLabel', 'notable work'), ('has', 'http://www.wikidata.org/prop/direct/P84'), ('hasLabel', 'architect'), ('work', 'http://www.wikidata.org/entity/Q5604'), ('workLabel', 'Frank Lloyd Wright')]
[('p', 'http://www.wikidata.org/prop/direct/P800'), ('pLabel', 'notable work'), ('has', 'http://www.wikidata.org/prop/direct/P131'), ('hasLabel', 'located in the administrative territorial entity'), ('wor

7

In [29]:
# the name of one of the artist's works is demonstrated
queryString = """

    SELECT DISTINCT ?work ?workLabel 
    WHERE{
      wd:Q5604 ?p wd:Q2081276 .
      wd:Q55276305 ?has ?work .
                  
                  
    ## labels

     ?has <http://schema.org/name> ?hasLabel .    
     ?work <http://schema.org/name> ?workLabel .    
           
    }
    LIMIT 10

"""

print("Results")
run_query(queryString)

Results
[('work', 'http://www.wikidata.org/entity/Q2081276'), ('workLabel', 'Pettit Memorial Chapel')]
[('work', 'http://www.wikidata.org/entity/Q4167836'), ('workLabel', 'Wikimedia category')]
[('work', 'http://www.wikidata.org/entity/Q55267305'), ('workLabel', 'Commons categories: recently created wikidata items 2')]


3

In [30]:

# here let's check the works of this artist to see how to grab its works and buildings

queryString = """

    SELECT DISTINCT ?name ?nameLabel WHERE{
      wd:Q5604 ?p wd:Q2081276 .
      wd:Q2081276 ?has ?work .
      ?work rdfs:label ?name .
    ## labels
         ?name <http://schema.org/name> ?nameLabel .
         

    }
    LIMIT 50
"""

print("Results")
run_query(queryString)

Results
Empty


0

In [31]:
# showing the name of the architecture's building

queryString = """

    SELECT DISTINCT ?americanarLabel (GROUP_CONCAT(DISTINCT ?workLabel; SEPARATOR=", ") AS ?workLabels)
     WHERE{
      ?americanar wdt:P27 wd:Q30  . 
      ?americanar wdt:P106 wd:Q42973 .
      ?work wdt:P84 ?americanar .
      
    ## labels
         ?americanar <http://schema.org/name> ?americanarLabel .
         ?work <http://schema.org/name> ?workLabel .

    }
    LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('americanarLabel', 'Nathan Clifford Ricker'), ('workLabels', 'Altgeld Hall')]
[('americanarLabel', 'Donn Barber'), ('workLabels', 'Hartford Times Building, Terminal Station, Travelers Tower')]
[('americanarLabel', 'Francis Costigan'), ('workLabels', 'Charles L. Shrewsbury House, Lanier Mansion, Le Méridien Indianapolis Hotel')]
[('americanarLabel', 'Albert C. Martin Jr.'), ('workLabels', '1900 Avenue of the Stars')]
[('americanarLabel', 'Frank E. Edbrooke'), ('workLabels', 'Central Presbyterian Church, Joslin Dry Goods Company Building, Masonic Temple Building, Steamboat Springs Depot, Temple Emanuel, The Brown Palace Hotel and Spa, Autograph Collection')]
[('americanarLabel', 'Félix Candela'), ('workLabels', "L'Oceanogràfic, Palacio de los Deportes, church of Nuestra Señora de Guadalupe")]
[('americanarLabel', 'James R. Edmunds, Jr.'), ('workLabels', 'Eastern High School, Maryland Stadium')]
[('americanarLabel', 'Gottlieb Schumacher'), ('workLabels', 'Carmel Winery')]
[('ame

100

In [32]:


queryString = """

    SELECT DISTINCT ?americanar ?americanarLabel ?workLabel ?typebuilding ?typebuildingLabel ?architectural_style ?architectural_styleLabel WHERE{
      ?americanar wdt:P27 wd:Q30  . 
      ?americanar wdt:P106 wd:Q42973 .
      ?work wdt:P84 ?americanar .
                  
      # here we have to add something for gaining the arts 
      ?art wdt:P31 ?typebuilding ;
                  wdt:P149 ?architectural_style .    
                  
    ## labels
         ?americanar <http://schema.org/name> ?americanarLabel .
         ?work  <http://schema.org/name> ?workLabel .
         ?typebuilding  <http://schema.org/name> ?typebuildingLabel . 
         ?architectural_style <http://schema.org/name> ?architectural_styleLabel .

    }
    LIMIT 40
    
"""

print("Results")
run_query(queryString)




Results
[('americanar', 'http://www.wikidata.org/entity/Q13647155'), ('americanarLabel', 'Marshall Strabala'), ('workLabel', 'Burj Khalifa'), ('typebuilding', 'http://www.wikidata.org/entity/Q18756633'), ('typebuildingLabel', 'garden shed'), ('architectural_style', 'http://www.wikidata.org/entity/Q2479493'), ('architectural_styleLabel', 'eclectic architecture')]
[('americanar', 'http://www.wikidata.org/entity/Q13647155'), ('americanarLabel', 'Marshall Strabala'), ('workLabel', 'Burj Khalifa'), ('typebuilding', 'http://www.wikidata.org/entity/Q18814970'), ('typebuildingLabel', 'retail building with apartments'), ('architectural_style', 'http://www.wikidata.org/entity/Q478742'), ('architectural_styleLabel', 'Amsterdam School')]
[('americanar', 'http://www.wikidata.org/entity/Q13647155'), ('americanarLabel', 'Marshall Strabala'), ('workLabel', 'Burj Khalifa'), ('typebuilding', 'http://www.wikidata.org/entity/Q21092604'), ('typebuildingLabel', 'manor house'), ('architectural_style', 'http:

40

In [33]:

queryString = """

    SELECT DISTINCT ?americanarLabel ?workLabel ?typebuildingLabel ?architectural_styleLabel WHERE{
      ?americanar wdt:P27 wd:Q30  . 
      ?americanar wdt:P106 wd:Q42973 .
      ?work wdt:P84 ?americanar .
                  
      # here we have to add something for gaining the arts 
      ?art wdt:P31 ?typebuilding ;
                  wdt:P149 ?architectural_style .    
                  
    ## labels
         ?americanar <http://schema.org/name> ?americanarLabel .
         ?work  <http://schema.org/name> ?workLabel .
         ?typebuilding  <http://schema.org/name> ?typebuildingLabel . 
         ?architectural_style <http://schema.org/name> ?architectural_styleLabel
    }
    LIMIT 40
    
"""

print("Results")
run_query(queryString)

Results
[('americanarLabel', 'Marshall Strabala'), ('workLabel', 'Burj Khalifa'), ('typebuildingLabel', 'garden shed'), ('architectural_styleLabel', 'eclectic architecture')]
[('americanarLabel', 'Marshall Strabala'), ('workLabel', 'Burj Khalifa'), ('typebuildingLabel', 'retail building with apartments'), ('architectural_styleLabel', 'Amsterdam School')]
[('americanarLabel', 'Marshall Strabala'), ('workLabel', 'Burj Khalifa'), ('typebuildingLabel', 'manor house'), ('architectural_styleLabel', 'vernacular architecture')]
[('americanarLabel', 'Marshall Strabala'), ('workLabel', 'Burj Khalifa'), ('typebuildingLabel', 'national parish'), ('architectural_styleLabel', 'Romanesque Revival architecture')]
[('americanarLabel', 'Marshall Strabala'), ('workLabel', 'Burj Khalifa'), ('typebuildingLabel', 'konak'), ('architectural_styleLabel', 'Ottoman architecture')]
[('americanarLabel', 'Marshall Strabala'), ('workLabel', 'Burj Khalifa'), ('typebuildingLabel', 'Holy Sepulchre chapels'), ('architec

40

In [34]:
# architectural style
queryString = """
    SELECT ?fallingwater ?fallingwaterLabel ?o ?oLabel
    WHERE { 
        wd:Q463179 ?fallingwater ?o .
        
    ## labels
         ?fallingwater <http://schema.org/name> ?fallingwaterLabel .
         ?o <http://schema.org/name> ?oLabel .
    } 
    LIMIT 100


"""

print("Results")
run_query(queryString)


Results
[('fallingwater', 'http://www.wikidata.org/prop/direct/P31'), ('fallingwaterLabel', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q1307276'), ('oLabel', 'single-family detached home')]
[('fallingwater', 'http://www.wikidata.org/prop/direct/P149'), ('fallingwaterLabel', 'architectural style'), ('o', 'http://www.wikidata.org/entity/Q245188'), ('oLabel', 'modern architecture')]
[('fallingwater', 'http://www.wikidata.org/prop/direct/P463'), ('fallingwaterLabel', 'member of'), ('o', 'http://www.wikidata.org/entity/Q55065239'), ('oLabel', 'Iconic Houses Network')]
[('fallingwater', 'http://www.wikidata.org/prop/direct/P1435'), ('fallingwaterLabel', 'heritage designation'), ('o', 'http://www.wikidata.org/entity/Q1459900'), ('oLabel', 'Tentative World Heritage Site')]
[('fallingwater', 'http://www.wikidata.org/prop/direct/P17'), ('fallingwaterLabel', 'country'), ('o', 'http://www.wikidata.org/entity/Q30'), ('oLabel', 'United States of America')]
[('fallingwater', 'http://www.wi

14

In [35]:
# architectural style

queryString = """

    SELECT ?fallingwater ?fallingwaterLabel ?o ?oLabel ?style ?styleLabel
    WHERE { 
        wd:Q463179 ?fallingwater ?o .
        wd:Q463179 wdt:P149 ?style .
        
     ## labels
     ?fallingwater <http://schema.org/name> ?fallingwaterLabel .
     ?o <http://schema.org/name> ?oLabel .
     ?style <http://schema.org/name> ?styleLabel   
    } 
    LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('fallingwater', 'http://www.wikidata.org/prop/direct/P131'), ('fallingwaterLabel', 'located in the administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q1896250'), ('oLabel', 'Stewart Township'), ('style', 'http://www.wikidata.org/entity/Q245188'), ('styleLabel', 'modern architecture')]
[('fallingwater', 'http://www.wikidata.org/prop/direct/P138'), ('fallingwaterLabel', 'named after'), ('o', 'http://www.wikidata.org/entity/Q34038'), ('oLabel', 'waterfall'), ('style', 'http://www.wikidata.org/entity/Q245188'), ('styleLabel', 'modern architecture')]
[('fallingwater', 'http://www.wikidata.org/prop/direct/P1435'), ('fallingwaterLabel', 'heritage designation'), ('o', 'http://www.wikidata.org/entity/Q43113623'), ('oLabel', 'part of UNESCO World Heritage Site'), ('style', 'http://www.wikidata.org/entity/Q245188'), ('styleLabel', 'modern architecture')]
[('fallingwater', 'http://www.wikidata.org/prop/direct/P1435'), ('fallingwaterLabel', 'heritage designation'),

14

In [36]:
# architectural style

queryString = """

    SELECT DISTINCT ?style ?styleLabel
    WHERE { 
        wd:Q463179 ?fallingwater ?o .
        wd:Q463179 wdt:P149 ?style .
        
        ## labels
        ?fallingwater <http://schema.org/name> ?fallingwaterLabel .
        ?o <http://schema.org/name> ?oLabel .
        ?style <http://schema.org/name> ?styleLabel   

    } 
    LIMIT 200
"""

print("Results")
run_query(queryString)


Results
[('style', 'http://www.wikidata.org/entity/Q245188'), ('styleLabel', 'modern architecture')]


1

In [37]:
# type of the architecture 
#### finding the type

queryString = """

    SELECT ?s ?sLabel ?o ?oLabel
    WHERE { 
        wd:Q463179 ?s ?o .
         ## labels
         ?p <http://schema.org/name> ?pLabel .
         ?o <http://schema.org/name> ?oLabel .

    } 
    LIMIT 25
    

"""

print("Results")
run_query(queryString)
# wdt:P31	http://www.wikidata.org/prop/direct/P31	 wd:Q1307276	single-family detached home

Results
[('s', 'http://www.wikidata.org/prop/direct/P84'), ('o', 'http://www.wikidata.org/entity/Q5604'), ('oLabel', 'Frank Lloyd Wright')]
[('s', 'http://www.wikidata.org/prop/direct/P84'), ('o', 'http://www.wikidata.org/entity/Q5604'), ('oLabel', 'Frank Lloyd Wright')]
[('s', 'http://www.wikidata.org/prop/direct/P84'), ('o', 'http://www.wikidata.org/entity/Q5604'), ('oLabel', 'Frank Lloyd Wright')]
[('s', 'http://www.wikidata.org/prop/direct/P84'), ('o', 'http://www.wikidata.org/entity/Q5604'), ('oLabel', 'Frank Lloyd Wright')]
[('s', 'http://www.wikidata.org/prop/direct/P84'), ('o', 'http://www.wikidata.org/entity/Q5604'), ('oLabel', 'Frank Lloyd Wright')]
[('s', 'http://www.wikidata.org/prop/direct/P84'), ('o', 'http://www.wikidata.org/entity/Q5604'), ('oLabel', 'Frank Lloyd Wright')]
[('s', 'http://www.wikidata.org/prop/direct/P84'), ('o', 'http://www.wikidata.org/entity/Q5604'), ('oLabel', 'Frank Lloyd Wright')]
[('s', 'http://www.wikidata.org/prop/direct/P84'), ('o', 'http://www

25

In [38]:
#### finding the type and name of the building

queryString = """

    SELECT DISTINCT ?nameofbuilding ?nameofbuildingLabel ?typebuilding ?typebuildingLabel
    WHERE { 
        
        wd:Q463179 ?s ?o .
        wd:Q463179 wdt:P31 ?typebuilding .
        
            ## labels
     ?nameofbuilding <http://schema.org/name> ?nameofbuildingLabel .
     ?typebuilding <http://schema.org/name> ?typebuildingLabel .

    } 
    LIMIT 10
    
"""

print("Results")
run_query(queryString)




Results
[('nameofbuilding', 'http://www.wikidata.org/entity/P348'), ('nameofbuildingLabel', 'software version identifier'), ('typebuilding', 'http://www.wikidata.org/entity/Q1307276'), ('typebuildingLabel', 'single-family detached home')]
[('nameofbuilding', 'http://www.wikidata.org/entity/P4669'), ('nameofbuildingLabel', 'tabular software version'), ('typebuilding', 'http://www.wikidata.org/entity/Q1307276'), ('typebuildingLabel', 'single-family detached home')]
[('nameofbuilding', 'http://www.wikidata.org/entity/P747'), ('nameofbuildingLabel', 'has edition or translation'), ('typebuilding', 'http://www.wikidata.org/entity/Q1307276'), ('typebuildingLabel', 'single-family detached home')]
[('nameofbuilding', 'http://www.wikidata.org/entity/Q1002728'), ('nameofbuildingLabel', 'Fonsi Nieto'), ('typebuilding', 'http://www.wikidata.org/entity/Q1307276'), ('typebuildingLabel', 'single-family detached home')]
[('nameofbuilding', 'http://www.wikidata.org/entity/Q100326534'), ('nameofbuildingL

10

In [39]:
#### finding the type # architectural style
queryString = """

    SELECT DISTINCT ?typebuildingLabel ?style ?styleLabel
    WHERE { 
        wd:Q463179 wdt:P149 ?style .
        wd:Q463179 ?s ?o .
        wd:Q463179 wdt:P31 ?typebuilding .

        ## labels
     
        ?typebuilding <http://schema.org/name> ?typebuildingLabel .
        ?style <http://schema.org/name> ?styleLabel .

    } 
    LIMIT 10

"""

print("Results")
run_query(queryString)

Results
[('typebuildingLabel', 'single-family detached home'), ('style', 'http://www.wikidata.org/entity/Q245188'), ('styleLabel', 'modern architecture')]


1

In [40]:

#fianl
#?work ?s ?o .

queryString = """
    SELECT ?americanarLabel ?workLabel ?typebuildingLabel
    WHERE { 
        ?americanar wdt:P27 wd:Q30  . 
        ?americanar wdt:P106 wd:Q42973 .
        ?work wdt:P84 ?americanar .
        ?work wdt:P31 ?typebuilding .
       # ?work wdt:P149 ?style .
        
        
        ## labels
        ?americanar <http://schema.org/name> ?americanarLabel .
        ?work <http://schema.org/name> ?workLabel .
        ?typebuilding <http://schema.org/name> ?typebuildingLabel .

    } 
    LIMIT 100


"""

print("Results")
run_query(queryString)

Results
[('americanarLabel', 'David Adler'), ('workLabel', 'Mrs. Isaac D. Adler House'), ('typebuildingLabel', 'single-family detached home')]
[('americanarLabel', 'C. Howard Crane'), ('workLabel', 'World Theater'), ('typebuildingLabel', 'theatre')]
[('americanarLabel', 'C. Howard Crane'), ('workLabel', 'World Theater'), ('typebuildingLabel', 'movie theater')]
[('americanarLabel', 'C. Howard Crane'), ('workLabel', 'World Theater'), ('typebuildingLabel', 'destroyed building or structure')]
[('americanarLabel', 'Charles E. Choate'), ('workLabel', 'Tennille Banking Company Building'), ('typebuildingLabel', 'commercial building')]
[('americanarLabel', 'George I. Barnett'), ('workLabel', 'Samuel Moody Grubbs House'), ('typebuildingLabel', 'single-family detached home')]
[('americanarLabel', 'Henry Franklin Kilburn'), ('workLabel', 'Pabst Hotel'), ('typebuildingLabel', 'hotel')]
[('americanarLabel', 'Matthew Del Gaudio'), ('workLabel', 'Our Lady of Pompeii Church'), ('typebuildingLabel', 'na

100

In [41]:
# the names of the building
queryString = """
    SELECT DISTINCT ?americanarchitectLabel ?nameofbuilding ?typebuildingLabel ?styleLabel
    WHERE { 
        ?work wdt:P84 ?americanarchitect .
        ?americanarchitect wdt:P27 wd:Q30  . 
        ?americanarchitect wdt:P106 wd:Q42973 .
        ?work wdt:P31 ?typebuilding .
        ?work wdt:P149 ?style .
        
        
        
        ## labels
        ?work <http://schema.org/name> ?nameofbuilding .
        ?americanarchitect <http://schema.org/name> ?americanarchitectLabel .
        ?typebuilding <http://schema.org/name> ?typebuildingLabel .
        ?style <http://schema.org/name> ?styleLabel .

    } 
    LIMIT 50
    
"""

print("Results")
run_query(queryString)

Results
[('americanarchitectLabel', 'David Adler'), ('nameofbuilding', 'Mrs. Isaac D. Adler House'), ('typebuildingLabel', 'single-family detached home'), ('styleLabel', 'Colonial Revival architecture')]
[('americanarchitectLabel', 'Matthew Del Gaudio'), ('nameofbuilding', 'Our Lady of Pompeii Church'), ('typebuildingLabel', 'national parish'), ('styleLabel', 'Romanesque Revival architecture')]
[('americanarchitectLabel', 'Matthew Del Gaudio'), ('nameofbuilding', 'Our Lady of Pompeii Church'), ('typebuildingLabel', 'shrine'), ('styleLabel', 'Romanesque Revival architecture')]
[('americanarchitectLabel', 'Matthew Del Gaudio'), ('nameofbuilding', 'Our Lady of Pompeii Church'), ('typebuildingLabel', 'parish church'), ('styleLabel', 'Romanesque Revival architecture')]
[('americanarchitectLabel', 'Matthew Del Gaudio'), ('nameofbuilding', 'Our Lady of Pompeii Church'), ('typebuildingLabel', 'Catholic church building'), ('styleLabel', 'Romanesque Revival architecture')]
[('americanarchitectLa

50

# Identify the BGP that connect a building with its architect, type, and architectural style

In [42]:
# this is the final BGP that list all of the information needed.
queryString = """

    SELECT DISTINCT ?nameofbuilding ?americanarchitectLabel ?typebuildingLabel ?styleLabel
    WHERE { 
        ?work wdt:P84 ?americanarchitect .
        ?americanarchitect wdt:P27 wd:Q30  . 
        ?americanarchitect wdt:P106 wd:Q42973 .
        ?work wdt:P31 ?typebuilding .
        ?work wdt:P149 ?style .
       # ?work rdfs:label ?nameofbuilding .
        
        ## labels
        ?work <http://schema.org/name> ?nameofbuilding .
        ?americanarchitect <http://schema.org/name> ?americanarchitectLabel .
        ?typebuilding <http://schema.org/name> ?typebuildingLabel .
        ?style <http://schema.org/name> ?styleLabel .
    } 
    LIMIT 100

"""

print("Results")
run_query(queryString)


Results
[('nameofbuilding', 'Mrs. Isaac D. Adler House'), ('americanarchitectLabel', 'David Adler'), ('typebuildingLabel', 'single-family detached home'), ('styleLabel', 'Colonial Revival architecture')]
[('nameofbuilding', 'Our Lady of Pompeii Church'), ('americanarchitectLabel', 'Matthew Del Gaudio'), ('typebuildingLabel', 'national parish'), ('styleLabel', 'Romanesque Revival architecture')]
[('nameofbuilding', 'Our Lady of Pompeii Church'), ('americanarchitectLabel', 'Matthew Del Gaudio'), ('typebuildingLabel', 'shrine'), ('styleLabel', 'Romanesque Revival architecture')]
[('nameofbuilding', 'Our Lady of Pompeii Church'), ('americanarchitectLabel', 'Matthew Del Gaudio'), ('typebuildingLabel', 'parish church'), ('styleLabel', 'Romanesque Revival architecture')]
[('nameofbuilding', 'Our Lady of Pompeii Church'), ('americanarchitectLabel', 'Matthew Del Gaudio'), ('typebuildingLabel', 'Catholic church building'), ('styleLabel', 'Romanesque Revival architecture')]
[('nameofbuilding', 'F

100

# Third

##### Which architect is the most prolific in the U.S. and outside?

In [43]:
# 1. find the names of the building
# 2. the names of the architect + their works

queryString = """

    SELECT DISTINCT ?americanar ?americanarLabel ?notablework ?notableworkLabel
    WHERE { 
        ?americanar wdt:P27 wd:Q30  . 
        ?americanar wdt:P106 wd:Q42973 .
        ?americanar wdt:P800 ?notablework .
        # ?work wdt:P84 ?americanarchitect .
        
        ## labels
        ?americanar <http://schema.org/name> ?americanarLabel .
        ?notablework <http://schema.org/name> ?notableworkLabel .

    #  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
    } 
    LIMIT 100
    

"""

print("Results")
run_query(queryString)



Results
[('americanar', 'http://www.wikidata.org/entity/Q16023299'), ('americanarLabel', 'Ambrose J. Russell'), ('notablework', 'http://www.wikidata.org/entity/Q7971885'), ('notableworkLabel', "Washington Governor's Mansion")]
[('americanar', 'http://www.wikidata.org/entity/Q5006523'), ('americanarLabel', 'C. Howard Crane'), ('notablework', 'http://www.wikidata.org/entity/Q1130746'), ('notableworkLabel', 'Earls Court Exhibition Centre')]
[('americanar', 'http://www.wikidata.org/entity/Q3017360'), ('americanarLabel', 'David Adler'), ('notablework', 'http://www.wikidata.org/entity/Q5050132'), ('notableworkLabel', 'Castle Hill')]
[('americanar', 'http://www.wikidata.org/entity/Q5006523'), ('americanarLabel', 'C. Howard Crane'), ('notablework', 'http://www.wikidata.org/entity/Q1201545'), ('notableworkLabel', 'Detroit Olympia')]
[('americanar', 'http://www.wikidata.org/entity/Q20988921'), ('americanarLabel', 'Gertrude Sawyer'), ('notablework', 'http://www.wikidata.org/entity/Q5177248'), ('n

100

In [44]:
## count the works of each artists
# but first count the number of works by one of the artists

# the number of works by 'Frank Lloyd wright'

queryString = """

    SELECT (COUNT (DISTINCT ?work) AS ?countwork) WHERE{
      ?work wdt:P84 wd:Q5604 .
      
       ## labels
        #?p <http://schema.org/name> ?pLabel .
        #?o <http://schema.org/name> ?oLabel .

    }

"""

print("Results")
run_query(queryString)



Results
[('countwork', '244')]


1

In [45]:

### and join them with this one:


queryString = """

    SELECT DISTINCT ?americanarchitectLabel ?workLabel
    WHERE { 
        ?work wdt:P84 ?americanarchitect .
        ?americanarchitect wdt:P27 wd:Q30  . 
        ?americanarchitect wdt:P106 wd:Q42973 .
        
        
        ## labels
        ?americanarchitect <http://schema.org/name> ?americanarchitectLabel .
        ?work <http://schema.org/name> ?workLabel .
        
        
      # SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
    } 
    LIMIT 100

"""

print("Results")
run_query(queryString)








Results
[('americanarchitectLabel', 'David Adler'), ('workLabel', 'Mrs. Isaac D. Adler House')]
[('americanarchitectLabel', 'C. Howard Crane'), ('workLabel', 'World Theater')]
[('americanarchitectLabel', 'Charles E. Choate'), ('workLabel', 'Tennille Banking Company Building')]
[('americanarchitectLabel', 'George I. Barnett'), ('workLabel', 'Samuel Moody Grubbs House')]
[('americanarchitectLabel', 'Henry Franklin Kilburn'), ('workLabel', 'Pabst Hotel')]
[('americanarchitectLabel', 'Matthew Del Gaudio'), ('workLabel', 'Our Lady of Pompeii Church')]
[('americanarchitectLabel', 'Sheperd S. Woodcock'), ('workLabel', 'First Baptist Church of Scituate')]
[('americanarchitectLabel', 'Ivey P. Crutchfield'), ('workLabel', 'Peterson-Wilbanks House')]
[('americanarchitectLabel', 'C. Howard Crane'), ('workLabel', 'Earls Court Exhibition Centre')]
[('americanarchitectLabel', 'Marshall Strabala'), ('workLabel', 'Burj Khalifa')]
[('americanarchitectLabel', 'C. Howard Crane'), ('workLabel', 'August Wil

100

# Which architect is the most prolific in the U.S. and outside?

In [46]:
# final BGP


queryString = """
    SELECT ?americanarchitectLabel ?countwork WHERE {
      {

              SELECT ?americanarchitect (COUNT (DISTINCT ?work) AS ?countwork) WHERE
              {
              ?work wdt:P84 ?americanarchitect .
              ?americanarchitect wdt:P27 wd:Q30  . 
              ?americanarchitect wdt:P106 wd:Q42973 .
              # BIND (wdt:P84 AS ?prop) .
            
               }
        GROUP BY ?americanarchitect
      }  .
  ?americanarchitect <http://schema.org/name> ?americanarchitectLabel .
    }
   
    ORDER BY DESC(?countwork) ?americanarchitectLabel
    LIMIT 1
    #LIMIT 100

"""

print("Results")
run_query(queryString)

Results
[('americanarchitectLabel', 'Frank Lloyd Wright'), ('countwork', '244')]


1

In [47]:
# and the artists who have the fewest works are:


queryString = """

    SELECT ?americanarchitectLabel ?countwork WHERE {
      {

              SELECT ?americanarchitect (COUNT (DISTINCT ?work) AS ?countwork) WHERE
              {
              ?work ?prop ?americanarchitect .
              ?americanarchitect wdt:P27 wd:Q30  . 
              ?americanarchitect wdt:P106 wd:Q42973 .
               BIND (wdt:P84 AS ?prop) .
               }
        GROUP BY ?americanarchitect
      } . 
      ?americanarchitect <http://schema.org/name> ?americanarchitectLabel .
     # SERVICE wikibase:label {bd:serviceParam wikibase:language "en" .}
    }
    # HAVING (?count > 1)        ### use this to show only repeated values
    ORDER BY ASC(?countwork) ?americanarchitectLabel
    LIMIT 10


"""

print("Results")
run_query(queryString)







Results
[('americanarchitectLabel', 'A. Lawrence Kocher'), ('countwork', '1')]
[('americanarchitectLabel', 'A. T. Mann'), ('countwork', '1')]
[('americanarchitectLabel', 'A. Wynn Howell'), ('countwork', '1')]
[('americanarchitectLabel', 'A.E. Hanson'), ('countwork', '1')]
[('americanarchitectLabel', 'A.G. Bauer'), ('countwork', '1')]
[('americanarchitectLabel', 'Aaron Green'), ('countwork', '1')]
[('americanarchitectLabel', 'Abraham H. Albertson'), ('countwork', '1')]
[('americanarchitectLabel', 'Abraham Rogatnick'), ('countwork', '1')]
[('americanarchitectLabel', 'Abraham William Hajjar'), ('countwork', '1')]
[('americanarchitectLabel', 'Ada Bell Maescher'), ('countwork', '1')]


10

## How many countries are or have been monarchies across continents?

In [48]:

# FOURTH :
# Analyze the location of the buildings designed by the american architects
## Which styles exists in the U.S.? Which type of building?
## In which U.S. states there are more notable buildings?
## Which U.S. state contains the largest number of building designed by U.S. architects

# FOURTH: ONE

queryString = """
    SELECT DISTINCT ?americanar ?americanarLabel ?work ?workLabel ?city ?cityLabel
    WHERE { 
        ?americanar wdt:P27 wd:Q30  . 
        ?americanar wdt:P106 wd:Q42973 .
        ?work wdt:P84 ?americanar .
        ?work wdt:P131 ?city .
              
              
        ## labels    
        ?americanar <http://schema.org/name> ?americanarLabel .
        ?work <http://schema.org/name> ?workLabel .              
        ?city <http://schema.org/name> ?cityLabel .            
     #   SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
    }
    LIMIT 50
"""

print("Results")
run_query(queryString)








Results
[('americanar', 'http://www.wikidata.org/entity/Q29093464'), ('americanarLabel', 'Ivey P. Crutchfield'), ('work', 'http://www.wikidata.org/entity/Q29093460'), ('workLabel', 'Peterson-Wilbanks House'), ('city', 'http://www.wikidata.org/entity/Q2695904'), ('cityLabel', 'Vidalia')]
[('americanar', 'http://www.wikidata.org/entity/Q29093464'), ('americanarLabel', 'Ivey P. Crutchfield'), ('work', 'http://www.wikidata.org/entity/Q29093503'), ('workLabel', 'Leader–Rosansky House'), ('city', 'http://www.wikidata.org/entity/Q2695904'), ('cityLabel', 'Vidalia')]
[('americanar', 'http://www.wikidata.org/entity/Q29093464'), ('americanarLabel', 'Ivey P. Crutchfield'), ('work', 'http://www.wikidata.org/entity/Q28811758'), ('workLabel', 'Crawford W. Brazell House'), ('city', 'http://www.wikidata.org/entity/Q2695904'), ('cityLabel', 'Vidalia')]
[('americanar', 'http://www.wikidata.org/entity/Q5077062'), ('americanarLabel', 'Charles E. Choate'), ('work', 'http://www.wikidata.org/entity/Q16995712

50

In [49]:
# FOURTH: ONE

# the BGP from the first BGP :


queryString = """
    SELECT DISTINCT ?americanarLabel ?workLabel ?cityLabel
    WHERE { 
        ?americanar wdt:P27 wd:Q30  . 
        ?americanar wdt:P106 wd:Q42973 .
        ?work wdt:P84 ?americanar .
        ?work wdt:P131 ?city .
              
              
        ## labels    
        ?americanar <http://schema.org/name> ?americanarLabel .
        ?work <http://schema.org/name> ?workLabel .              
        ?city <http://schema.org/name> ?cityLabel .            
     #   SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
    }
    LIMIT 50
"""

print("Results")
run_query(queryString)






Results
[('americanarLabel', 'Ivey P. Crutchfield'), ('workLabel', 'Peterson-Wilbanks House'), ('cityLabel', 'Vidalia')]
[('americanarLabel', 'Ivey P. Crutchfield'), ('workLabel', 'Leader–Rosansky House'), ('cityLabel', 'Vidalia')]
[('americanarLabel', 'Ivey P. Crutchfield'), ('workLabel', 'Crawford W. Brazell House'), ('cityLabel', 'Vidalia')]
[('americanarLabel', 'Charles E. Choate'), ('workLabel', 'Holt Brothers Banking Company Building'), ('cityLabel', 'Sandersville')]
[('americanarLabel', 'Francis Willford Fitzpatrick'), ('workLabel', 'Johnston Memorial Building'), ('cityLabel', 'Wallace')]
[('americanarLabel', 'C. Howard Crane'), ('workLabel', 'World Theater'), ('cityLabel', 'Omaha')]
[('americanarLabel', 'Abraham Wesley Eager'), ('workLabel', 'Auditorium (Torrance High School)'), ('cityLabel', 'Torrance')]
[('americanarLabel', 'Charles E. Choate'), ('workLabel', 'James Kelley House'), ('cityLabel', 'Washington County')]
[('americanarLabel', 'Ambrose J. Russell'), ('workLabel', '

50

In [50]:
## the country predicate
queryString = """
    SELECT DISTINCT ?americanar ?americanarLabel ?work ?workLabel ?country ?countryLabel
    WHERE { 
        ?americanar wdt:P27 wd:Q30  . 
        ?americanar wdt:P106 wd:Q42973 .
        ?work wdt:P84 ?americanar .
        ?work wdt:P17 ?country .

        ## labels
        ?americanar <http://schema.org/name> ?americanarLabel .
        ?work <http://schema.org/name> ?workLabel .              
        #?city <http://schema.org/name> ?cityLabel .  
        ?country <http://schema.org/name> ?countryLabel .  
    } 
    LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('americanar', 'http://www.wikidata.org/entity/Q5006523'), ('americanarLabel', 'C. Howard Crane'), ('work', 'http://www.wikidata.org/entity/Q1130746'), ('workLabel', 'Earls Court Exhibition Centre'), ('country', 'http://www.wikidata.org/entity/Q145'), ('countryLabel', 'United Kingdom')]
[('americanar', 'http://www.wikidata.org/entity/Q5006523'), ('americanarLabel', 'C. Howard Crane'), ('work', 'http://www.wikidata.org/entity/Q23693569'), ('workLabel', 'Metropolitan Theatre'), ('country', 'http://www.wikidata.org/entity/Q16'), ('countryLabel', 'Canada')]
[('americanar', 'http://www.wikidata.org/entity/Q3017360'), ('americanarLabel', 'David Adler'), ('work', 'http://www.wikidata.org/entity/Q16198500'), ('workLabel', 'Mrs. Isaac D. Adler House'), ('country', 'http://www.wikidata.org/entity/Q30'), ('countryLabel', 'United States of America')]
[('americanar', 'http://www.wikidata.org/entity/Q5006523'), ('americanarLabel', 'C. Howard Crane'), ('work', 'http://www.wikidata.org/entity

25

In [51]:
# adding USA as country
queryString = """
    SELECT DISTINCT ?americanar ?americanarLabel ?work ?workLabel ?country ?countryLabel
    WHERE { 
        ?americanar wdt:P27 wd:Q30  . 
        ?americanar wdt:P106 wd:Q42973 .
        ?work wdt:P84 ?americanar ;
              wdt:P17 wd:Q30 .
             # BIND (wd:Q30 AS ?country) .
              
              
        ## labels
        ?americanar <http://schema.org/name> ?americanarLabel .
        ?work <http://schema.org/name> ?workLabel .              
        #?city <http://schema.org/name> ?cityLabel .  
        ?country <http://schema.org/name> ?countryLabel .  
    } 
    LIMIT 100

"""

print("Results")
run_query(queryString)


Results
[('americanar', 'http://www.wikidata.org/entity/Q3017360'), ('americanarLabel', 'David Adler'), ('work', 'http://www.wikidata.org/entity/Q16198500'), ('workLabel', 'Mrs. Isaac D. Adler House'), ('country', 'http://www.wikidata.org/entity/P348'), ('countryLabel', 'software version identifier')]
[('americanar', 'http://www.wikidata.org/entity/Q3017360'), ('americanarLabel', 'David Adler'), ('work', 'http://www.wikidata.org/entity/Q16198500'), ('workLabel', 'Mrs. Isaac D. Adler House'), ('country', 'http://www.wikidata.org/entity/P4669'), ('countryLabel', 'tabular software version')]
[('americanar', 'http://www.wikidata.org/entity/Q3017360'), ('americanarLabel', 'David Adler'), ('work', 'http://www.wikidata.org/entity/Q16198500'), ('workLabel', 'Mrs. Isaac D. Adler House'), ('country', 'http://www.wikidata.org/entity/P747'), ('countryLabel', 'has edition or translation')]
[('americanar', 'http://www.wikidata.org/entity/Q3017360'), ('americanarLabel', 'David Adler'), ('work', 'http

100

In [52]:
#Which styles exists in the U.S.? Which type of building?

queryString = """
    SELECT DISTINCT ?americanarLabel ?workLabel ?countryLabel ?styleLabel ?typebuildingLabel
    WHERE { 
        ?americanar wdt:P27 wd:Q30  . 
        ?americanar wdt:P106 wd:Q42973 .
        ?work wdt:P84 ?americanar ;
              wdt:P17 wd:Q30 .
        ?work wdt:P31 ?typebuilding .
        ?work wdt:P149 ?style .
        #BIND (wd:Q30 AS ?country) .
              
              
        ## labels
         ?americanar <http://schema.org/name> ?americanarLabel .
         ?work <http://schema.org/name> ?workLabel .              
         ?country <http://schema.org/name> ?countryLabel .
         ?style <http://schema.org/name> ?styleLabel .  
         ?typebuilding <http://schema.org/name> ?typebuildingLabel         

    } 
    LIMIT 50

"""

print("Results")
run_query(queryString)


Results
[('americanarLabel', 'David Adler'), ('workLabel', 'Mrs. Isaac D. Adler House'), ('countryLabel', 'software version identifier'), ('styleLabel', 'Colonial Revival architecture'), ('typebuildingLabel', 'single-family detached home')]
[('americanarLabel', 'David Adler'), ('workLabel', 'Mrs. Isaac D. Adler House'), ('countryLabel', 'tabular software version'), ('styleLabel', 'Colonial Revival architecture'), ('typebuildingLabel', 'single-family detached home')]
[('americanarLabel', 'David Adler'), ('workLabel', 'Mrs. Isaac D. Adler House'), ('countryLabel', 'has edition or translation'), ('styleLabel', 'Colonial Revival architecture'), ('typebuildingLabel', 'single-family detached home')]
[('americanarLabel', 'David Adler'), ('workLabel', 'Mrs. Isaac D. Adler House'), ('countryLabel', 'Fonsi Nieto'), ('styleLabel', 'Colonial Revival architecture'), ('typebuildingLabel', 'single-family detached home')]
[('americanarLabel', 'David Adler'), ('workLabel', 'Mrs. Isaac D. Adler House'),

50

# Which styles exists in the U.S.? Which type of building?

In [53]:
queryString = """
SELECT DISTINCT ?styleLabel ?typebuildingLabel
WHERE { 
        ?americanar wdt:P27 wd:Q30  . 
        ?americanar wdt:P106 wd:Q42973 .
        ?work wdt:P84 ?americanar ;
              wdt:P17 wd:Q30 .
        ?work wdt:P31 ?typebuilding .
        ?work wdt:P149 ?style .

              
        ## labels

         ?style <http://schema.org/name> ?styleLabel .  
         ?typebuilding <http://schema.org/name> ?typebuildingLabel .      
} 
LIMIT 5
"""

print("Results")
run_query(queryString)


Results
[('styleLabel', 'Tudor architecture'), ('typebuildingLabel', 'hospital')]
[('styleLabel', 'Tudor architecture'), ('typebuildingLabel', 'former hospital')]
[('styleLabel', 'Art Deco architecture'), ('typebuildingLabel', 'skyscraper')]
[('styleLabel', 'Neoclassical architecture'), ('typebuildingLabel', 'single-family detached home')]
[('styleLabel', 'Neoclassical architecture'), ('typebuildingLabel', 'capitol building')]


5

In [54]:
### NOTE: THIS is the style and type by all architects who made a work in USA.
#  (ALL ARCHITECTS)
queryString = """
    SELECT DISTINCT ?styleLabel ?typebuildingLabel
    WHERE { 
        ?americanar wdt:P106 wd:Q42973 .
        ?work wdt:P84 ?americanar ;
              wdt:P17 wd:Q30 .
        ?work wdt:P31 ?typebuilding .
        ?work wdt:P149 ?style .

              
        ## labels
         ?americanar <http://schema.org/name> ?americanarLabel .
         ?work <http://schema.org/name> ?workLabel .              
         ?country <http://schema.org/name> ?countryLabel .
         ?style <http://schema.org/name> ?styleLabel .  
         ?typebuilding <http://schema.org/name> ?typebuildingLabel .    

    } 
    LIMIT 5

"""

print("Results")
run_query(queryString)


Results
[('styleLabel', 'Colonial Revival architecture'), ('typebuildingLabel', 'single-family detached home')]
[('styleLabel', 'Romanesque Revival architecture'), ('typebuildingLabel', 'national parish')]
[('styleLabel', 'Romanesque Revival architecture'), ('typebuildingLabel', 'shrine')]
[('styleLabel', 'Romanesque Revival architecture'), ('typebuildingLabel', 'parish church')]
[('styleLabel', 'Romanesque Revival architecture'), ('typebuildingLabel', 'Catholic church building')]


5

In [55]:
# cities
queryString = """
    SELECT DISTINCT ?city ?cityLabel ?partof ?partofLabel ?p ?pLabel ?o ?oLabel
    WHERE { 
        ?americanar wdt:P106 wd:Q42973 .
        ?work wdt:P84 ?americanar .
        ?work wdt:P31 ?typebuilding .
        ?work wdt:P149 ?style .
        ?work wdt:P131 ?city ;
              wdt:P17 wd:Q30 .
        ?partof wdt:P31 ?city .
        ?partof ?p ?o .

        #BIND (wd:Q30 AS ?country) .
      
        ## labels        
         #?typebuilding <http://schema.org/name> ?typebuildingLabel   
         ?city <http://schema.org/name> ?cityLabel   
         #?partof <http://schema.org/name> ?partofLabel
         #?p <http://schema.org/name> ?pLabel
         #?o <http://schema.org/name> ?oLabel
     
     #   SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
    } 
    LIMIT 50

"""

print("Results")
run_query(queryString)




Results
[('city', 'http://www.wikidata.org/entity/Q65'), ('cityLabel', 'Los Angeles'), ('partof', 'http://www.wikidata.org/entity/Q3491216'), ('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('o', 'http://wikiba.se/ontology#Item')]
[('city', 'http://www.wikidata.org/entity/Q65'), ('cityLabel', 'Los Angeles'), ('partof', 'http://www.wikidata.org/entity/Q3491216'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('o', 'http://www.wikidata.org/entity/Q65')]
[('city', 'http://www.wikidata.org/entity/Q65'), ('cityLabel', 'Los Angeles'), ('partof', 'http://www.wikidata.org/entity/Q3491216'), ('p', 'http://www.wikidata.org/prop/direct/P360'), ('o', 'http://www.wikidata.org/entity/Q37073')]
[('city', 'http://www.wikidata.org/entity/Q65'), ('cityLabel', 'Los Angeles'), ('partof', 'http://www.wikidata.org/entity/Q3491216'), ('p', 'http://schema.org/name'), ('o', 'list of Lab Rats episodes')]
[('city', 'http://www.wikidata.org/entity/Q16552'), ('cityLabel', 'San Diego'), ('partof', 'ht

50

In [56]:
### cities
## find the city of Pettit Memorial Chapel. 

queryString = """

    SELECT ?p ?pLabel ?o ?oLabel
    WHERE { 
        wd:Q2081276 ?p ?o .
        
        
        ## labels
         ?p <http://schema.org/name> ?pLabel .
         ?o <http://schema.org/name> ?oLabel .
    }
    LIMIT 100
"""

print("Results")
run_query(queryString)




Results
[('p', 'http://www.wikidata.org/prop/direct/P149'), ('pLabel', 'architectural style'), ('o', 'http://www.wikidata.org/entity/Q245188'), ('oLabel', 'modern architecture')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pLabel', 'country'), ('o', 'http://www.wikidata.org/entity/Q30'), ('oLabel', 'United States of America')]
[('p', 'http://www.wikidata.org/prop/direct/P1435'), ('pLabel', 'heritage designation'), ('o', 'http://www.wikidata.org/entity/Q19558910'), ('oLabel', 'National Register of Historic Places listed place')]
[('p', 'http://www.wikidata.org/prop/direct/P84'), ('pLabel', 'architect'), ('o', 'http://www.wikidata.org/entity/Q5604'), ('oLabel', 'Frank Lloyd Wright')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pLabel', 'located in the administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q185609'), ('oLabel', 'Belvidere')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pLabel', "topic's main category"), ('o', 'http://www.wik

7

In [57]:
## find the city of Pettit Memorial Chapel. 
# wdt:P131	http://www.wikidata.org/prop/direct/P131	 wd:Q185609	Belvidere from previous one
# wdt:P131 is for its city, 
queryString = """

    SELECT ?o ?oLabel
    WHERE { 
        wd:Q2081276 wdt:P131 ?o .

        ## labels
         ?p <http://schema.org/name> ?pLabel .
         ?o <http://schema.org/name> ?oLabel .
    }
    LIMIT 5

"""

print("Results")
run_query(queryString)




Results
[('o', 'http://www.wikidata.org/entity/Q185609'), ('oLabel', 'Belvidere')]
[('o', 'http://www.wikidata.org/entity/Q185609'), ('oLabel', 'Belvidere')]
[('o', 'http://www.wikidata.org/entity/Q185609'), ('oLabel', 'Belvidere')]
[('o', 'http://www.wikidata.org/entity/Q185609'), ('oLabel', 'Belvidere')]
[('o', 'http://www.wikidata.org/entity/Q185609'), ('oLabel', 'Belvidere')]


5

### In which U.S. states there are more notable buildings?

In [58]:
# all architects (NOT JUST AMERICAN Architects)
# all countries
queryString = """
    SELECT ?city ?cityLabel ?countwork WHERE {
      {

              SELECT ?city (COUNT (DISTINCT ?work) AS ?countwork) WHERE
              {
              ?work wdt:P84 ?architect .
              ?work wdt:P131 ?city .
              ?work wdt:P17 ?country .
              ?architect wdt:P106 wd:Q42973 .
              # BIND (wd:Q30 AS ?country) .
               }
        GROUP BY ?city
      } . 
    ## labels
     ?city <http://schema.org/name> ?cityLabel .
    }


    ORDER BY DESC(?countwork) ?cityLabel 
    LIMIT 10


"""

print("Results")
run_query(queryString)


Results
[('city', 'http://www.wikidata.org/entity/Q1757'), ('cityLabel', 'Helsinki'), ('countwork', '1070')]
[('city', 'http://www.wikidata.org/entity/Q2807'), ('cityLabel', 'Madrid'), ('countwork', '786')]
[('city', 'http://www.wikidata.org/entity/Q11299'), ('cityLabel', 'Manhattan'), ('countwork', '643')]
[('city', 'http://www.wikidata.org/entity/Q9899'), ('cityLabel', 'Amsterdam'), ('countwork', '495')]
[('city', 'http://www.wikidata.org/entity/Q1741'), ('cityLabel', 'Vienna'), ('countwork', '459')]
[('city', 'http://www.wikidata.org/entity/Q506250'), ('cityLabel', 'Stockholm Municipality'), ('countwork', '413')]
[('city', 'http://www.wikidata.org/entity/Q1492'), ('cityLabel', 'Barcelona'), ('countwork', '403')]
[('city', 'http://www.wikidata.org/entity/Q490'), ('cityLabel', 'Milan'), ('countwork', '398')]
[('city', 'http://www.wikidata.org/entity/Q973974'), ('cityLabel', 'Prague 1'), ('countwork', '388')]
[('city', 'http://www.wikidata.org/entity/Q649'), ('cityLabel', 'Moscow'), ('

10

In [59]:
#  just buildings in USA by all architects (NOT JUST AMERICAN Architects)
queryString = """


    SELECT ?city ?cityLabel ?countwork WHERE {
      {

              SELECT ?city (COUNT (DISTINCT ?work) AS ?countwork) WHERE
              {
              ?work wdt:P84 ?architect .
              ?work wdt:P131 ?city .
              ?work wdt:P17 wd:Q30 .
              ?architect wdt:P106 wd:Q42973 .
              
               }
        GROUP BY ?city
      } . 
        ## labels
     ?city <http://schema.org/name> ?cityLabel .
    }


    ORDER BY DESC(?countwork) ?cityLabel 
    LIMIT 10


"""

print("Results")
run_query(queryString)


Results
[('city', 'http://www.wikidata.org/entity/Q11299'), ('cityLabel', 'Manhattan'), ('countwork', '643')]
[('city', 'http://www.wikidata.org/entity/Q61'), ('cityLabel', 'Washington, D.C.'), ('countwork', '225')]
[('city', 'http://www.wikidata.org/entity/Q1384'), ('cityLabel', 'New York'), ('countwork', '192')]
[('city', 'http://www.wikidata.org/entity/Q771'), ('cityLabel', 'Massachusetts'), ('countwork', '173')]
[('city', 'http://www.wikidata.org/entity/Q1297'), ('cityLabel', 'Chicago'), ('countwork', '170')]
[('city', 'http://www.wikidata.org/entity/Q99'), ('cityLabel', 'California'), ('countwork', '137')]
[('city', 'http://www.wikidata.org/entity/Q1546'), ('cityLabel', 'Iowa'), ('countwork', '128')]
[('city', 'http://www.wikidata.org/entity/Q1397'), ('cityLabel', 'Ohio'), ('countwork', '125')]
[('city', 'http://www.wikidata.org/entity/Q1439'), ('cityLabel', 'Texas'), ('countwork', '121')]
[('city', 'http://www.wikidata.org/entity/Q6106'), ('cityLabel', 'Portland'), ('countwork', 

10

## Which U.S. state contains the largest number of building designed by U.S. architects

In [60]:

queryString = """
    SELECT ?cityLabel ?countwork WHERE {
      {

              SELECT ?city (COUNT (DISTINCT ?work) AS ?countwork) WHERE
              {
              ?work wdt:P84 ?americanarchitect .
              ?work wdt:P131 ?city .
              ?work wdt:P17 wd:Q30 .
              ?americanarchitect wdt:P27 wd:Q30  . 
              ?americanarchitect wdt:P106 wd:Q42973 .
               # BIND (wd:Q30 AS ?country) .
               }

        GROUP BY ?city
      } . 
    ## labels                           
      ?city <http://schema.org/name> ?cityLabel .
    }
    ORDER BY DESC(?countwork) ?cityLabel 
    LIMIT 20

"""

print("Results")
run_query(queryString)




Results
[('cityLabel', 'Manhattan'), ('countwork', '599')]
[('cityLabel', 'Washington, D.C.'), ('countwork', '202')]
[('cityLabel', 'New York'), ('countwork', '189')]
[('cityLabel', 'Massachusetts'), ('countwork', '159')]
[('cityLabel', 'Chicago'), ('countwork', '158')]
[('cityLabel', 'Iowa'), ('countwork', '127')]
[('cityLabel', 'California'), ('countwork', '119')]
[('cityLabel', 'Texas'), ('countwork', '119')]
[('cityLabel', 'Ohio'), ('countwork', '115')]
[('cityLabel', 'Pennsylvania'), ('countwork', '112')]
[('cityLabel', 'Portland'), ('countwork', '109')]
[('cityLabel', 'Florida'), ('countwork', '84')]
[('cityLabel', 'Connecticut'), ('countwork', '78')]
[('cityLabel', 'Illinois'), ('countwork', '78')]
[('cityLabel', 'Los Angeles'), ('countwork', '70')]
[('cityLabel', 'Oregon'), ('countwork', '66')]
[('cityLabel', 'Detroit'), ('countwork', '65')]
[('cityLabel', 'San Francisco'), ('countwork', '63')]
[('cityLabel', 'Philadelphia'), ('countwork', '58')]
[('cityLabel', 'Maine'), ('coun

20